In [43]:
import pandas as pd 
import re
from datasets import Dataset

In [44]:
## Load nartuo series transcripts.

data_path = '../data/naruto.csv'
 
transcript = pd.read_csv(data_path)

In [45]:
transcript.head(10)

,name,line
0,Naruto,(Laughing) Give it up. (Shows the stone faces...
1,Hiruzen,(Turns away from his writing) I hope you’re n...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah
5,Iruka,"(In Naruto’s ear) Oh yeah, Naruto!?"
6,Naruto,"Where’d you come from, Iruka Sensei!? What ar..."
7,Iruka,"(Points at Naruto) No, what are you doing her..."
8,Iruka,"(Arms folded) I’m at the end of my rope, Naru..."
9,Sakura,"Alright, Sakura here. Let’s do it. Transform!..."


In [46]:
## Remove actions from transcripts leveraging regular expressions. (regex)
def remove_parenthesis(text):
    return re.sub(r'\(.*?\)', '', text)

In [47]:
transcript['line'] = transcript['line'].apply(remove_parenthesis)
transcript

,name,line
0,Naruto,"Give it up. You’re just bent, because you d..."
1,Hiruzen,I hope you’re not bothering me with some tri...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah
...,...,...
158,Iruka,Congratulations. You graduate. Naruto’s stand...
159,Iruka,Huh?
160,Naruto,Iruka Sensei!
161,Iruka,Ah! That hurts!


In [48]:
transcript['no_of_words'] = transcript['line'].str.strip().str.split(' ')
transcript['no_of_words'] = transcript['no_of_words'].apply(lambda x: len(x))

transcript['no_of_words'].head(10)

0    26
1    16
2     2
3     6
4     1
5     3
6    11
7    12
8    43
9     7
Name: no_of_words, dtype: int64

In [49]:
transcript['naturo_response_flag'] = 0
transcript.loc[ (transcript['name'] == 'Naruto')&(transcript['no_of_words'] > 5), 'naturo_response_flag']=1
transcript.head(10)

,name,line,no_of_words,naturo_response_flag
0,Naruto,"Give it up. You’re just bent, because you d...",26,1
1,Hiruzen,I hope you’re not bothering me with some tri...,16,0
2,Ninja,Naseer Sabah,2,0
3,Ninja,is the best person on earth,6,0
4,Naruto,muah,1,0
5,Iruka,"Oh yeah, Naruto!?",3,0
6,Naruto,"Where’d you come from, Iruka Sensei!? What ar...",11,1
7,Iruka,"No, what are you doing here? You’re supposed...",12,0
8,Iruka,"I’m at the end of my rope, Naruto. You faile...",43,0
9,Sakura,"Alright, Sakura here. Let’s do it. Transform!",7,0


In [50]:
indicies = list(transcript[(transcript['naturo_response_flag']== 1)&(transcript.index > 0)].index)
indicies[:3]

[6, 28, 30]

In [51]:
## System Prompt
system_prompt = ''' Your role is to assimilate the role of the character named "Naruto" from the renowned anime serioes "Naruto". Your responses, when prompted, should reflect that of his personality and his speech patterns to best 
imitate his likeliness.\n '''

prompts = []

for index in indicies:

    prompt = system_prompt

    prompt += transcript.iloc[index -1]['line']
    prompt += '\n'
    prompt += transcript.iloc[index]['line']

    prompts.append(prompt)

print(prompts[3])

 Your role is to assimilate the role of the character named "Naruto" from the renowned anime serioes "Naruto". Your responses, when prompted, should reflect that of his personality and his speech patterns to best 
imitate his likeliness.
  Well, I was just thinking. Maybe after you clean this all up I can take you out for some ramen. The good stuff. What do you think? 
  Now that’s some serious motivation! I’ll have this clean in no time!


In [52]:
## Process into dataframes for better model digestion.

dataframe = pd.DataFrame({'prompt' : prompts})
dataframe.head()

,prompt
0,Your role is to assimilate the role of the ch...
1,Your role is to assimilate the role of the ch...
2,Your role is to assimilate the role of the ch...
3,Your role is to assimilate the role of the ch...
4,Your role is to assimilate the role of the ch...


In [53]:
dataset = Dataset.from_pandas(dataframe)